# On the Meaning and Use of Singular Value Decomposition

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import train_test_split

## Eigendecomposition

Let's start with eigendecomposition. Remember PCA?

Any *non-defective* and *square* matrix $A$ has an eigendecomposition:

$\large A = Q\Lambda Q^{-1}$,

where:

- the columns of $Q$ are the eigenvectors of $A$, and
- $\Lambda$ is a diagonal matrix whose non-zero entries are the eigenvalues of $A$.

Eigendecompositions have *many* practical applications.

But since not all matrices are square, not all matrices have eigendecompositions.

## Singular Value Decomposition

However, given a non-square matrix $R$, we can construct a square matrix by simply calculating $RR^T$ or $R^TR$.

(The 'T' superscript indicates that the relevant matrix is *transposed*, i.e. its rows and columns are switched. So for example the transpose of $\begin{bmatrix} 0 & 1 & 2 \\ 3 & 4 & 5 \\ 6 & 7 & 8 \end{bmatrix}$ is $\begin{bmatrix} 0 & 3 & 6 \\ 1 & 4 & 7 \\ 2 & 5 & 8 \end{bmatrix}$.)

Moreover, _any_ matrix $A$ has a ***singular value decomposition***, i.e. a factorization in the form:

$\large A = U\Sigma V^T$,

where

- $\Sigma$ is diagonal if square and otherwise "pseudo-diagonal" (a diagonal matrix sitting on top of zeroes) with real, non-negative entries; and
- $U$ and $V$ are orthogonal.

A matrix $Q$ is orthogonal if its columns are mutually orthogonal and normalized to lengths of 1. This guarantees that, for orthogonal $Q$, $Q^TQ = I$. Thus, $Q^T = Q^T(QQ^{-1}) = (Q^TQ)Q^{-1}$, so $Q^T = Q^{-1}$.

Note also that, if $V$ is orthogonal, then so is $V^{-1}$:

$(V^{-1})^T = (V^T)^T$ <br/>
$(V^{-1})^T = V$ <br/>
$(V^{-1})^T = (V^{-1})^{-1}$

Now (this text adapted from [Inderjit Dhillon]( http://www.cs.utexas.edu/users/inderjit/courses/dm2009/LinearAlgebraBackground.pdf)):

Using the singular value decomposition of $A$, we have:

$\large AA^T = U\Sigma V^T\times(U\Sigma V^T)^T$ <br/>
$\large AA^T = U\Sigma V^T\times V\Sigma^TU^T$ <br/>
$\large AA^T = U\Sigma I\Sigma U^T$ <br/>
$\large AA^T = U\Sigma^2U^T$ <br/>
Here we have an eigendecomposition of $AA^T$ in terms of the SVD of $A$!

In particular:

the eigenvectors of $AA^T$ are the columns of $U$; and <br/>
the eigenvalues of $AA^T$ are the squares of the singular values of $A$. <br/>

Similarly, $\large A^TA = V\Sigma^2V^T$.

And so:

the eigenvectors of $A^TA$ are the columns of $V$; and <br/>
the eigenvalues of $A^TA$ are the squares of the singular values of $A$.<br/>

Put another way: The singular values of A are the non-negative square roots of the eigenvalues of $AA^T$ or $A^TA$.

Again (see [this page](https://math.stackexchange.com/questions/2152751/why-does-the-eigenvalues-of-aat-are-the-squares-of-the-singular-values-of-a)):

Since: <br/> $\large AA^T = U\Sigma^2U^T$, <br/> we have that <br/> $\large AA^TU = U\Sigma^2$ (since $U$ is orthogonal), <br/> which says that $AA^T$ multiplied by a vector (let's choose $U_i$, a column of $U$) yields $U$ multiplied by a scalar, namely $\sigma^2_i$. I.e. the squares of the singular values of $A$ are the (non-zero) eigenvalues of $A^TA$ (or $AA^T$).

### Eigenvalues of $AA^T$ and $A^TA$
Why do $AA^T$ and $A^TA$ have the same eigenvalues?

Let $\lambda$ be a (non-zero) eigenvalue of $A^TA$.

Then:

$A^TAx = \lambda x$.

Now let $Ax = y$. So we have:

$A^Ty = \lambda x$. If we left-multiply by $A$, we have:

$AA^Ty = A\lambda x = \lambda Ax = \lambda y$, which is to say that $\lambda$ is an eigenvalue of $AA^T$.

(To show the reverse, just let $B = A^T$.)

For more, see [this post](https://math.stackexchange.com/questions/1087064/non-zero-eigenvalues-of-aat-and-ata).

### Diagonalization vs. SVD

This shows that the eigen- and the singular value decompositions are intimately related!

The SVD has many uses! Check out [this book chapter](https://www.cs.cmu.edu/~venkatg/teaching/CStheory-infoage/book-chapter-4.pdf) for details.

### SVD and Dimensionality Reduction

The SVD, much like PCA, can be used to reduce the dimensionality of your data.

Recall how PCA works: We start with an eigendecomposition of our covariance matrix. We then order our eigenvectors by the size of their corresponding eigenvalues. Eigenvectors with large eigenvalues explain more of the variance in our dataset, and so we define our principal components accordingly.

The situation is much the same with the SVD. The singular vectors that correspond to larger singular values capture more of the variance in our data. And, just as with PCA, we can often capture a large percentage of the variance by taking a relatively small number of singular vectors, throwing out the ones that correspond to small singular values. [Here](https://rpubs.com/Tanmay007/svd) is a good example of this process.

## SVD in Python

Let's show that the squares of the singular values of a non-square matrix $A$ are equal to the eigenvalues of $A^TA$ (or $AA^T$).

In [ ]:
np.random.seed(42)
A = np.random.rand(5, 3)
A

In [ ]:
# Using np.linalg.svd()

np.linalg.svd(A)

`np.linalg.svd()` returns a triple, unsurprisingly: The left singular vectors ("U"), the singular values, and the (transpose of the) right singular vectors ("V").

We can re-create A by multiplying these components together. But we'll first have to turn the array of singular values into $\Sigma$, so we'll use `np.diag()` together with `np.vstack()`.

In [ ]:
u, s, vT = np.linalg.svd(A)

In [ ]:
sigma = np.vstack([np.diag(s), [[0, 0, 0], [0, 0, 0]]])

In [ ]:
sigma

In [ ]:
# This should reproduce the matrix A

u.dot(sigma).dot(vT)

Now let's look at the eigendecomposition of $AA^T$.

In [ ]:
np.linalg.eig(A.dot(A.T))

Let's extract the eigenvalues.

In [ ]:
np.linalg.eig(A.dot(A.T))[0]

The last two eigenvalues are really equal to *zero*, which of course can often confuse computers. Squaring the singular values of $A$ should yield the same list (without the zeroes):

In [ ]:
np.linalg.svd(A)[1]**2

##  Least-Squares Problem

The singular value decomposition can be used to solve a least-squares problem quickly. Let's create such a problem.

### Comparison to the matrix-vector equation, $M\vec{x} = \vec{b}$

Suppose we have an exact equation, $M\vec{x} = \vec{b}$.

In that case $M$ is square, and the solution to the equation is $x = M^{-1}b$.

In [ ]:
np.random.seed(43)
M = np.random.rand(5, 5)
b = np.random.rand(5, 1)

In [ ]:
b

In [ ]:
x = np.linalg.inv(M).dot(b)

In [ ]:
# Reproducing the vector b

M.dot(x)

### Optimization Problem

But of course the typical DS situation is that we have not an exact equation to solve but rather an optimization to perform. So let's now imagine that $A$ has more rows than columns.

If we need some warm and fuzzy familiarity, we could throw this all into a DataFrame:

In [ ]:
pd.DataFrame(np.hstack([A, b]),
             columns=['pred1', 'pred2', 'pred3', 'target'])

Treating the columns of $A$ as our predictors and $b$ as our target, the answer to this least-squares problem turns out to be $A^+\vec{b}$, where $A^+$ is the *pseudo-inverse* of $A$. The formula for the pseudo-inverse if $(A^TA)^{-1}A^T$, and the idea behind it is to generalize the notion of an inverse to non-square matrices. The pseudo-inverse reduces to the inverse in the case of square matrices.

In [ ]:
mat = np.random.rand(100, 100)
np.allclose(np.linalg.inv(mat), np.linalg.pinv(mat))

If we have $A = U\Sigma V^T$, then $A^+ = V\Sigma^+U^T$. Numpy has a pseudo-inverse function, `np.linalg.pinv()`, which we could use directly, rather than first constructing the SVD. But because the decomposed equation involves only the pseudo-inverse of a (pseudo-) diagonal matrix, the SVD route can be much *faster*. **This is the real point of using the SVD in calculating least-squares solutions.**

See [this site](https://math.stackexchange.com/questions/974193/why-does-svd-provide-the-least-squares-and-least-norm-solution-to-a-x-b) for a proof of the least-squares solution. For more on the pseudo-inverse, see [here](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse).

In [ ]:
# Let's calculate the least-squares solution using our SVD components:

vT.T.dot(np.linalg.pinv(sigma)).dot(u.T).dot(b)

In [ ]:
# Checking against sklearn's LinearRegression():

LinearRegression(fit_intercept=False).fit(A, b).coef_

In fact, `LinearRegression()` uses SVD under the hood!

### Timings

In [ ]:
%timeit vT.T.dot(np.linalg.pinv(sigma)).dot(u.T).dot(b)

In [ ]:
%timeit LinearRegression(fit_intercept=False).fit(A, b).coef_

In [ ]:
%timeit np.linalg.pinv(A).dot(b)

For our small sample matrix, `sklearn`'s version actually takes longer. But let's try a much larger matrix!

In [ ]:
np.random.seed(42)
X = np.random.rand(10000, 100)
target = np.random.rand(10000, 1)

In [ ]:
%timeit np.linalg.pinv(X).dot(target)

In [ ]:
%timeit LinearRegression(fit_intercept=False).fit(X, target).coef_

Going through the SVD (as `sklearn`'s model-fitting procedure does) makes noticeable savings in time here.

## SVD in Rec Systems

Since the SVD solves the least-squares problem, it can be used in ALS-style recommenders. But whereas ALS is generally used for implicit ratings, the SVD is more useful for explicit ratings, such as exist in the [MovieLens](https://grouplens.org/datasets/movielens/) datset.

Much of the following code comes from [this](https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/) helpful article.

In [ ]:
data = pd.read_csv('data/ratings.dat',
                   names=['user_id', 'movie_id', 'rating', 'time'],
                   engine='python', delimiter='::')
data.head()

In [ ]:
data.tail()

In [ ]:
data.groupby('user_id').count()

In [ ]:
data.groupby('movie_id').count()

In [ ]:
movies = pd.read_csv('data/movies.dat',
                     names=['movie_id', 'title', 'genre'],
                    engine='python', delimiter='::')
movies.head()

In [ ]:
movies.tail()

In [ ]:
users = pd.read_csv('data/users.dat',
                    names=['user_id', 'gender', 'age',
                          'occupation', 'ZIP code'],
                   engine='python', delimiter='::')
users.head()

In [ ]:
users.tail()

Check the documentation for how the `age` and `occupation` columns are coded.

In [ ]:
# Creating the rating matrix (rows as movies, columns as users)

ratings_mat = np.ndarray(
    shape=(np.max(data['movie_id']), np.max(data['user_id'])),
    dtype=np.uint8)
ratings_mat[data['movie_id']-1, data['user_id']-1] = data['rating']

In [ ]:
data['movie_id'][0], data['user_id'][0]

In [ ]:
ratings_mat[1192, 0]

In [ ]:
ratings_mat[1096, 6039]

In [ ]:
# Normalizing the matrix (subtract mean movie rating off)

normalized_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, axis=1))]).T

For more on this type of "normalization" see [this post](https://stats.stackexchange.com/questions/31096/how-do-i-use-the-svd-in-collaborative-filtering) and [this article](http://nicolas-hug.com/blog/matrix_facto_3).

In [ ]:
# Computing the Singular Value Decomposition (SVD)

A = normalized_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

In [ ]:
def top_cosine_similarity(data, movie_id, top_n=10):
    """This function calculates cosine similarity and returns the
    top n highest scores for user-defined n."""
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    
    # This is just a shortcut for calculating vector norms:
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indices = np.argsort(-similarity)
    return sort_indices[:top_n]

In [ ]:
def print_similar_movies(movie_data, movie_id, top_indices):
    """This function prints out the movies corresponding to
    a set of indices"""
    print(f"""Recommendations for
    {movie_data[movie_data['movie_id'] == movie_id]['title'][0]}: \n""")
    for id in top_indices + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

In [ ]:
# k-principal components to represent movies,
# movie_id to find recommendations, top_n print n results      

k = 50
movie_id = 1 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indices = top_cosine_similarity(sliced, movie_id, top_n)

# Printing the top N similar movies
print_similar_movies(movie_data, movie_id, indices)

## The `surprise` package
`pip install surprise`

In [ ]:
# This code taken from https://surprise.readthedocs.io/en/stable/

# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

In [ ]:
predictions[:10]